In [ ]:
[전처리 순서 (미완)]

1. 다운받은 영상 데이터 불러오기
2. 영상 데이터(mp4)를 음성 데이터(wav)로 전환시킨 후에 따로 저장하기 -> data_1
3. 영상 데이터에서 소리 제거한 후에 따로 저장하기 -> data_2
4. data_1 파일 텍스트로 변환시켜서 단어 단위로 컷팅하고, 컷팅한 위치를 리스트 안에 저장하기
5. 음성 컷팅 위치와 data2를 대응시킨 후에 그 부분을 컷팅하여 따로 저장하기
    -> data_3
6. data_3 파일에서 하관/입술 부분만 크롭한 형태로 따로 저장하기 -> data_4

## 1. 영상 데이터 불러오기

In [1]:
import os
import re

In [2]:
# [필수] 파일 주소 재설정 
MOV_Directory = '../1. Crawling/MOV' 
FileList = os.listdir(MOV_Directory)
print(FileList)

['readnaversid1355oid056aid0000003463.mp4', 'readnaversid1355oid056aid0000003464.mp4', 'readnaversid1355oid056aid0000003465.mp4', 'readnaversid1355oid056aid0000003466.mp4', 'readnaversid1355oid056aid0000003467.mp4', 'readnaversid1355oid056aid0000003468.mp4', 'readnaversid1355oid056aid0000003469.mp4', 'readnaversid1355oid056aid0000003470.mp4', 'readnaversid1355oid056aid0000003471.mp4', 'readnaversid1355oid056aid0000003472.mp4', 'readnaversid1355oid056aid0000003474.mp4', 'readnaversid1355oid056aid0000003475.mp4', 'readnaversid1355oid056aid0000003476.mp4', 'readnaversid1355oid056aid0000003477.mp4', 'readnaversid1355oid056aid0000003478.mp4', 'readnaversid1355oid056aid0000003480.mp4', 'readnaversid1355oid056aid0000003481.mp4', 'readnaversid1355oid056aid0000003514.mp4', 'readnaversid1355oid056aid0000003515.mp4', 'readnaversid1355oid056aid0000003516.mp4', 'readnaversid1355oid056aid0000003517.mp4', 'readnaversid1355oid056aid0000003518.mp4', 'readnaversid1355oid056aid0000003519.mp4', 'readnaver

## 2. 영상 파일을 음성 파일로 전환

In [3]:
# !pip install moviepy
import moviepy.editor as mp

In [4]:
for i in FileList:
    i2 = MOV_Directory + '/' + i
    print(i2)
    clip = mp.VideoFileClip(i2)
    
    # [필수] 파일 주소 재설정 
    WAV_Directory = './WAV' 
    sound_file_name = re.sub(pattern='mp4', repl='wav', string=i)
    print(sound_file_name)
    sound_file_name = WAV_Directory + '/' + sound_file_name
    clip.audio.write_audiofile(sound_file_name)

../1. Crawling/MOV/readnaversid1355oid056aid0000003463.mp4


chunk:   3%|█▋                                                            | 67/2550 [00:00<00:03, 669.07it/s, now=None]

readnaversid1355oid056aid0000003463.wav
MoviePy - Writing audio in ./WAV/readnaversid1355oid056aid0000003463.wav


MoviePy - Done.
../1. Crawling/MOV/readnaversid1355oid056aid0000003464.mp4


chunk:   3%|█▉                                                            | 91/2868 [00:00<00:03, 909.79it/s, now=None]

...

MoviePy - Done.
../1. Crawling/MOV/readnaversid1355oid056aid0000003470.mp4


chunk:   1%|▋                                                             | 30/2535 [00:00<00:08, 299.93it/s, now=None]

readnaversid1355oid056aid0000003470.wav
MoviePy - Writing audio in ./WAV/readnaversid1355oid056aid0000003470.wav


chunk:  76%|█████████████████████████████████████████████              | 1936/2535 [00:00<00:00, 1645.75it/s, now=None]

KeyboardInterrupt: 